In [1]:
import chipwhisperer as cw
try:
    if not scope.connectStatus:
        scope.con()
except NameError:
    scope = cw.scope()
   
try:
    target = cw.target(scope)
except IOError:
    print("INFO: Caught exception on reconnecting to target - attempting to reconnect to scope first.")
    print("INFO: This is a work-around when USB has died without Python knowing. Ignore errors above this line.")
    scope = cw.scope()
    target = cw.target(scope)

print("INFO: Found ChipWhisperer😍\n")

Traceback (most recent call last):
  File "<ipython-input-1-536a38a260a6>", line 3, in <module>
    if not scope.connectStatus:
NameError: name 'scope' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/ms/chipwhisperer/software/chipwhisperer/hardware/naeusb/naeusb.py", line 295, in txrx
    response = self.open(serial_number=payload)
  File "/home/ms/chipwhisperer/software/chipwhisperer/hardware/naeusb/naeusb.py", line 349, in open
    dev.set_configuration()
  File "/home/ms/.local/lib/python3.7/site-packages/usb/core.py", line 869, in set_configuration
    self._ctx.managed_set_configuration(self, configuration)
  File "/home/ms/.local/lib/python3.7/site-packages/usb/core.py", line 102, in wrapper
    return f(self, *args, **kwargs)
  File "/home/ms/.local/lib/python3.7/site-packages/usb/core.py", line 148, in managed_set_configuration
    self.backend.set_configuration(self.handle, cfg.bConfiguration

Warning: Could not connect to "NewAE USB (CWLite/CW1200)". It may have been disconnected, is in an error state, or is being used by another tool.

In [ ]:
if "STM" in PLATFORM or PLATFORM == "CWLITEARM" or PLATFORM == "CWNANO":
    prog = cw.programmers.STM32FProgrammer
elif PLATFORM == "CW303" or PLATFORM == "CWLITEXMEGA":
    prog = cw.programmers.XMEGAProgrammer
else:
    prog = None

In [1]:
import time
time.sleep(0.05)
scope.default_setup()
def reset_target(scope):
    if PLATFORM == "CW303" or PLATFORM == "CWLITEXMEGA":
        scope.io.pdic = 'low'
        time.sleep(0.1)
        scope.io.pdic = 'high_z' #XMEGA doesn't like pdic driven high
        time.sleep(0.1) #xmega needs more startup time
        #Flush garbage too
        target.flush()
    # Reset pin in OpenTitan layout has to be connected to pdic pin of ChipWhisperer 
    if PLATFORM == "OpenTitan":
        scope.io.pdic = 'low'
        time.sleep(0.05)
        scope.io.pdic = 'high'
        time.sleep(0.05)
        #Flush garbage too
        target.flush()
    else:  
        scope.io.nrst = 'low'
        time.sleep(0.05)
        scope.io.nrst = 'high'
        time.sleep(0.05)
        #Flush garbage too
        target.flush()

NameError: name 'scope' is not defined

In [28]:
# Regular expression
import re
# Serial communication
import serial

CRED = '\033[91m'
CEND = '\033[0m'
CSTYLE = '\x1b[5;37;41m'
CBLACK  = '\33[30m'
CWHITE  = '\33[37m'
CORG = '\x1b[6;39;43m'

def connectSerial(target, baud = 230400):
    target_name = '/dev/ttyUSB'
    p = subprocess.Popen("ls -l /dev/ttyUSB*", shell=True, stdout=subprocess.PIPE)
    result = str(p.communicate()[0])
    s1 = ""
    s2 = ""

    try:
        m = re.search('/dev/ttyUSB(.*)', result)
        if m:
            found = m.group(1)
            target_name += found
            tar_baud = baud

            try:
                targets = target_name.split('\\n')
                device = targets[0]
                dev_id = device.split('/dev/ttyUSB')[1]
                #print("dev_id: ", dev_id)
                p = subprocess.Popen("udevadm info -a -n /dev/ttyUSB" + dev_id + "| grep -e \"idProduct\" -e \"product\" | head -n2", shell=True, stdout=subprocess.PIPE)
                result1 = str(p.communicate()[0])

                #s1 = ''.join(x for x in idProduct if x.isdigit()) # get idProduct from usb device
                #print("device: %s, idProduct: %s" % (device, s1))
                #print("Res1: ", result1)
                i = result1.find('{product}==')
                if i != -1:
                    t = result1.split('{product}==')
                    product1 = t[1] # result1[i+11:]
                    #print("product1: ", product1)
                    i = result1.find('{idProduct}==')
                    s1 = result1[i+14:i+18]
                    #print("device: %s, product: %s, idProduct: %s" % (device, product, s1))
                
                m = re.search('/dev/ttyUSB(.*)', targets[1])
                if m:
                    dev_id = m.group(1)
                    device2 = '/dev/ttyUSB' + dev_id
                    p = subprocess.Popen("udevadm info -a -n /dev/ttyUSB" + dev_id + "| grep -e \"idProduct\" -e \"product\" | head -n2", shell=True, stdout=subprocess.PIPE)
                    result2 = str(p.communicate()[0])
                    #print(result2)
                    d = result2.find('{product}==')
                    if d != -1:
                        t2 = result2.split('{product}==')
                        product2 = t2[1] # result2[i+11:]
                        #print("product2: ", product2)
                        d = result2.find('{idProduct}==')
                        s2 = result2[d+14:d+18]
                        #print("device2: %s, idProduct: %s" % (device2, 2))
                    #product2 = result2.split('{product}==')
                    #idProduct2 = result2.split('{idProduct}==')
                    #s2 = ''.join(x for x in idProduct2 if x.isdigit()) # get idProduct from usb device
                    #print("device2: %s, idProduct: %s" % (device2, s2))
                    
            except:
                pass
            
            target_devname = ""
            if (s1) == "6010": # this has to fit to Arty S7
                target_devname = device
                prod = product1
            elif s2 == "6010":
                target_devname = device2
                prod = product2

            
            if target_devname:
                try:
                    target.ser = serial.Serial(target_devname, tar_baud, timeout=250)
                except AttributeError:
                    print(CSTYLE + CBLACK + """Could not connect to """ + device + CEND)
    except AttributeError:
        print(CSTYLE + CBLACK + """Did not found a serial device.
        Check connection & the actual target_name on your own via "ls -l /dev/ttyUSB*" """ + CEND)
        # Alternatively, if this fails check it actual target_name on your own via ls -l /dev/ttyUSB* 

    if target.ser.port == target_devname:
        print("INFO: Opened port %s (%s) with speed %d\n" % (target.ser.port, prod, target.ser.baudrate))

In [1]:
def save_picture(folder, project, filename, figure):
    cnt = 1
    
    if project is None:
        print("Please define a project name for saving")
    else:
        if folder is None:
            folder = "/home/ms/chipwhisperer/traces/"
        else:            
            file =  folder + project + '/' + filename + '_0' + str(cnt) +'.png'
            my_file = Path(file)
            
            while my_file.is_file(): #  os.path.exists(file_name)
                # file exists
                cnt+=1
                file =  folder + project + '/' + filename + '_0' + str(cnt) +'.png'
                my_file = Path(file)
                
            print("Saved trace to: %s\n" % file)
            figure.savefig(file, quality=95, bbox_inches='tight')